In [8]:
#%% library import
import numpy as np
import pandas as pd
import networkx as nx
import torch as tc
import pickle
import time
import os
import subprocess

import pymysql
pymysql.install_as_MySQLdb()
import _mysql

from torch.autograd import Variable
from sklearn.utils import shuffle
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from functools import partial
from sklearn.model_selection import KFold
from sqlalchemy import create_engine

In [9]:
DB_HOST = "192.168.0.50"
PORT = 3306  #포트번호
USER_NAME = "gh506015"
USER_PSWD = "2281"
DB_NAME="binding_db"


class CallDB:
    def __init__(self):
        self.engine = create_engine("mysql+mysqldb://{}:{}@{}/{}".format(USER_NAME, USER_PSWD, DB_HOST, DB_NAME), encoding='utf-8')
    
    def query_db(self, query):
        dbconn = _mysql.connect(host=DB_HOST, user=USER_NAME, passwd=USER_PSWD, port=PORT, db=DB_NAME)
        self.cursor = dbconn.cursor()
        self.cursor.execute(query)
        dbconn.commit()
        dbconn.close()
    
    def to_db(self, data, table_nm):
        df = pd.DataFrame(data)
        df.to_sql(name=table_nm, con=self.engine, if_exists='append', index=False)
        
    def from_db(self, query):
        dbconn = _mysql.connect(host=DB_HOST, user=USER_NAME, passwd=USER_PSWD, port=PORT, db=DB_NAME)
        self.cursor = dbconn.cursor()
        self.cursor.execute(query)
        table = self.cursor.fetchall()
        dbconn.commit()
        dbconn.close()
        return pd.DataFrame(table)

In [42]:
db = CallDB()
pr_query = "select uniprot_id, protein_seq from kiba_protein "
li_query = "select chembl_id, smiles from kiba_ligand_db"
kiba_query = "select * from kiba_score "

sequence_df = db.from_db(pr_query)
smiles_df = db.from_db(li_query)
kiba_score = db.from_db(kiba_query)
datalen = len(kiba_score)
datalen

118254

In [38]:
sequence_dic = {row[1][0]: row[1][1] for row in sequence_df.iterrows()}
smiles_dic = {row[1][0]: row[1][1] for row in smiles_df.iterrows()}
del sequence_df, smiles_df

{'O00141': 'MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNSYACKHPEVQSILKISQPQEPELMNANPSPPPSPSQQINLGPSSNPHAKPSDFHFLKVIGKGSFGKVLLARHKAEEVFYAVKVLQKKAILKKKEEKHIMSERNVLLKNVKHPFLVGLHFSFQTADKLYFVLDYINGGELFYHLQRERCFLEPRARFYAAEIASALGYLHSLNIVYRDLKPENILLDSQGHIVLTDFGLCKENIEHNSTTSTFCGTPEYLAPEVLHKQPYDRTVDWWCLGAVLYEMLYGLPPFYSRNTAEMYDNILNKPLQLKPNITNSARHLLEGLLQKDRTKRLGAKDDFMEIKSHVFFSLINWDDLINKKITPPFNPNVSGPNDLRHFDPEFTEEPVPNSIGKSPDSVLVTASVKEAAEAFLGFSYAPPTDSFL',
 'O00311': 'MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEKLYEAVPQLSNVFKIEDKIGEGTFSSVYLATAQLQVGPEEKIALKHLIPTSHPIRIAAELQCLTVAGGQDNVMGVKYCFRKNDHVVIAMPYLEHESFLDILNSLSFQEVREYMLNLFKALKRIHQFGIVHRDVKPSNFLYNRRLKKYALVDFGLAQGTHDTKIELLKFVQSEAQQERCSQNKSHIITGNKIPLSGPVPKELDQQSTTKASVKRPYTNAQIQIKQGKDGKEGSVGLSVQRSVFGERNFNIHSSISHESPAVKLMKQSKTVDVLSRKLATKKKAISTKVMNSAVMRKTASSCPASLTCDCYATDKVCSICLSRRQQVAPRAGTPGFRAPEVLTKCPNQTTAIDMWSAGVIFLSLLSGRYPFYKASDDLTALAQIMTIRGSRETIQAAKTFGKSILCSKEVPAQDLRKLCERLRGMDSSTPKLTSDIQGHASHQPAISEKTDHKASCLVQTPPGQYSGNSFKKGDSNSCEHCFDEYNTNLEGWNEVPDEA

In [44]:
sequence = np.zeros((datalen, 4128))
smiles = np.zeros((datalen, 590))

for i, u in enumerate(kiba_score.iloc[:,0]):
    sequence[i] = sequence_dic[u]

ValueError: could not convert string to float: 'MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNSYACKHPEVQSILKISQPQEPELMNANPSPPPSPSQQINLGPSSNPHAKPSDFHFLKVIGKGSFGKVLLARHKAEEVFYAVKVLQKKAILKKKEEKHIMSERNVLLKNVKHPFLVGLHFSFQTADKLYFVLDYINGGELFYHLQRERCFLEPRARFYAAEIASALGYLHSLNIVYRDLKPENILLDSQGHIVLTDFGLCKENIEHNSTTSTFCGTPEYLAPEVLHKQPYDRTVDWWCLGAVLYEMLYGLPPFYSRNTAEMYDNILNKPLQLKPNITNSARHLLEGLLQKDRTKRLGAKDDFMEIKSHVFFSLINWDDLINKKITPPFNPNVSGPNDLRHFDPEFTEEPVPNSIGKSPDSVLVTASVKEAAEAFLGFSYAPPTDSFL'

In [20]:
datalen

118254

In [4]:
class Data_loader:
    def __init__(self):
        self.protein = 0    # #!!!
        self.ligand = 0
        self.score = 0
        
    def maketable(self):
        self.protein = self.protein.iloc['protein_seq']
        
        revised_dataset = list(zip(sequence, smileseq, kiba))
        shuffled_dataset = np.array(shuffle(revised_dataset)); del revised_dataset

#%% Load dataset and cuda
dataset = pd.read_csv("datasets/KIBA.csv")
datalen = len(dataset)

Using device: cuda
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [5]:
#%% protein-ligand-kiba split
protein = dataset.loc[:, "uniprotID"]    # 5
ligand = dataset.loc[:, "chemblID"]
kiba = list(dataset.loc[:, 'KIBA'])
del dataset


In [6]:
#%% protein sequence load
f = open('datasets/dictionaries/prt_lstm.txt', 'rb')
seq_voc, _ = pickle.load(f)
f.close()

sequence = np.zeros((datalen, 4128))
for i, s in enumerate(protein):
    sequence[i] = seq_voc[s]

sequence = sequence[:, :1400]


In [7]:
#%% ligand smiles load
f = open('datasets/dictionaries/lgn_smiecoding.txt', 'rb')
smi_dic = pickle.load(f)
f.close()

smileseq = np.zeros((datalen, 590))
for i, e in enumerate(ligand):
    smileseq[i] = smi_dic[e]

smileseq = smileseq[:, :100]


In [8]:
#%% dataset zip
revised_dataset = list(zip(sequence, smileseq, kiba))
shuffled_dataset = np.array(shuffle(revised_dataset)); del revised_dataset


In [9]:
#%% Make collate func.
def collate(samples):
    sequences, smileseq, labels = map(list, zip(*samples))
    return tc.LongTensor(sequences).cuda(), tc.LongTensor(smileseq).cuda(), tc.tensor(labels).cuda()

In [10]:
#%% learning module 선언
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()    # method 상속받고 __init__()은 여기서 하겠다.
        
        self.prt_emlayer = nn.Embedding(21, 10)
        
        self.prt_cv1dlayers = nn.Sequential(
                        nn.Conv1d(10, 32, kernel_size = 4),
                        nn.BatchNorm1d(num_features = 32),
                        nn.ReLU(),
                        nn.Conv1d(32, 64, kernel_size = 8),
                        nn.BatchNorm1d(num_features = 64),
                        nn.ReLU(),
                        nn.Conv1d(64, 96, kernel_size = 12),
                        nn.BatchNorm1d(num_features = 96),
                        nn.ReLU(),
                        nn.MaxPool1d(kernel_size=1379)
                        )
        
        ######################################################################
        ######################################################################
        
        self.lgn_emlayer = nn.Embedding(64, 10)
        
        self.lgn_cv1dlayers = nn.Sequential(
                        nn.Conv1d(10, 32, kernel_size = 4),
                        nn.BatchNorm1d(num_features = 32),
                        nn.ReLU(),
                        nn.Conv1d(32, 64, kernel_size = 6),
                        nn.BatchNorm1d(num_features = 64),
                        nn.ReLU(),
                        nn.Conv1d(64, 96, kernel_size = 8),
                        nn.BatchNorm1d(num_features = 96),
                        nn.ReLU(),
                        nn.MaxPool1d(kernel_size = 85)
                        )
        
        
        self.mlplayers = nn.Sequential(
                        nn.Linear(192, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(1024, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
                        nn.Dropout(0.1),
                        nn.Linear(512, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU()
                        )

        self.regress = nn.Linear(512, 1)    # regression

    def forward(self, prt_seq, lgn_seq):   
        p = self.prt_emlayer(prt_seq)
        p = p.permute(0, 2, 1)
        p = self.prt_cv1dlayers(p)
        p = p.squeeze()
        
        l = self.lgn_emlayer(lgn_seq)
        l = l.permute(0, 2, 1)
        l = self.lgn_cv1dlayers(l)
        l = l.squeeze()
        
        cat = tc.cat((p, l), axis=1).cuda()
        out = self.mlplayers(cat)
        
        return self.regress(out).cuda()

In [13]:
#%% Set hyperparameter
hp_d = {}

# FIXME: 학습 관련 하이퍼파라미터
hp_d['batch_size'] = 512
hp_d['num_epochs'] = 100

hp_d['init_learning_rate'] = 10 ** -3.09
hp_d['eps'] = 10 ** -9
hp_d['weight_decay'] = 10 ** -5.257

In [ ]:
#%% training and validation
kf = KFold(n_splits=5); kf.get_n_splits(shuffled_dataset)
val_err = 0
for tr_idx, ts_idx in kf.split(shuffled_dataset):
    trainset, testset = shuffled_dataset[tr_idx], shuffled_dataset[ts_idx]
    trainset = tuple(trainset); testset = tuple(testset)

    tr_data_loader = DataLoader(trainset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)
    va_data_loader = DataLoader(testset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)

    model = Regressor().to(tc.device('cuda:0'))
    loss_func = nn.MSELoss(reduction='mean').cuda()

    optimizer = optim.Adam(model.parameters(), lr=hp_d['init_learning_rate'], 
        weight_decay=hp_d['weight_decay'], eps=hp_d['eps'])

    tr_epoch_losses = []
    va_epoch_losses = []

    for epoch in range(hp_d['num_epochs']):                             #!! epoch-loop
        # training session
        model.train()
        tr_epoch_loss = 0

        for iter, (seq, smi, label) in enumerate(tr_data_loader):       #!! batch-loop
            prediction = model(seq, smi).view(-1).cuda()
            loss = loss_func(prediction, label).cuda()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tr_epoch_loss += loss.detach().item()

        tr_epoch_loss /= (iter + 1)
        print('Training epoch {}, loss {:.4f}'.format(epoch, tr_epoch_loss))
        tr_epoch_losses.append(tr_epoch_loss)

    # ===========================================================================
        # validation session
        model.eval()
        va_epoch_loss = 0

        for iter, (seq, smi, label) in enumerate(va_data_loader):  # batch-loop
            prediction = model(seq, smi).view(-1).cuda()
            loss = loss_func(prediction, label).cuda()

            va_epoch_loss += loss.detach().item()

        va_epoch_loss /= (iter + 1)
        print('Validation epoch {}, loss {:.4f}'.format(epoch, va_epoch_loss))
        va_epoch_losses.append(va_epoch_loss)
    val_err += min(va_epoch_losses)
print('10-fold CVMSE:', val_err/10)


Training epoch 0, loss 14.3761
Validation epoch 0, loss 0.4538
Training epoch 1, loss 0.6442
Validation epoch 1, loss 0.4708
Training epoch 2, loss 0.5404
Validation epoch 2, loss 0.6433
Training epoch 3, loss 0.4303
Validation epoch 3, loss 0.4040
Training epoch 4, loss 0.3549
Validation epoch 4, loss 0.3184
Training epoch 5, loss 0.3163
Validation epoch 5, loss 0.2910
Training epoch 6, loss 0.2823
Validation epoch 6, loss 0.2952
Training epoch 7, loss 0.2655
Validation epoch 7, loss 0.2778
Training epoch 8, loss 0.2578
Validation epoch 8, loss 0.2993
Training epoch 9, loss 0.2394
Validation epoch 9, loss 0.3038
Training epoch 10, loss 0.2296
Validation epoch 10, loss 0.2528
Training epoch 11, loss 0.2118
Validation epoch 11, loss 0.2451
Training epoch 12, loss 0.2047
Validation epoch 12, loss 0.2580
Training epoch 13, loss 0.2031
Validation epoch 13, loss 0.3933
Training epoch 14, loss 0.2052
Validation epoch 14, loss 0.2516
Training epoch 15, loss 0.1869
Validation epoch 15, loss 0.

Training epoch 29, loss 0.1138
Validation epoch 29, loss 0.1987
Training epoch 30, loss 0.1141
Validation epoch 30, loss 0.2427
Training epoch 31, loss 0.1122
Validation epoch 31, loss 0.2315
Training epoch 32, loss 0.1082
Validation epoch 32, loss 0.2253
Training epoch 33, loss 0.1023
Validation epoch 33, loss 0.2117
Training epoch 34, loss 0.1044
Validation epoch 34, loss 0.1930
Training epoch 35, loss 0.0990
Validation epoch 35, loss 0.2057
Training epoch 36, loss 0.0949
Validation epoch 36, loss 0.1835
Training epoch 37, loss 0.0975
Validation epoch 37, loss 0.1978
Training epoch 38, loss 0.0959
Validation epoch 38, loss 0.2498
Training epoch 39, loss 0.0938
Validation epoch 39, loss 0.2306
Training epoch 40, loss 0.0879
Validation epoch 40, loss 0.2090
Training epoch 41, loss 0.0850
Validation epoch 41, loss 0.1883
Training epoch 42, loss 0.0821
Validation epoch 42, loss 0.2068
Training epoch 43, loss 0.1329
Validation epoch 43, loss 0.2792
Training epoch 44, loss 0.1251
Validatio

Validation epoch 57, loss 0.1716
Training epoch 58, loss 0.0763
Validation epoch 58, loss 0.1827
Training epoch 59, loss 0.0718
Validation epoch 59, loss 0.1822
Training epoch 60, loss 0.0748
Validation epoch 60, loss 0.1787
Training epoch 61, loss 0.0784
Validation epoch 61, loss 0.1852
Training epoch 62, loss 0.0725
Validation epoch 62, loss 0.1809
Training epoch 63, loss 0.0675
Validation epoch 63, loss 0.1831
Training epoch 64, loss 0.0658
Validation epoch 64, loss 0.1641
Training epoch 65, loss 0.0645
Validation epoch 65, loss 0.1722
Training epoch 66, loss 0.0664
Validation epoch 66, loss 0.1956
Training epoch 67, loss 0.0699
Validation epoch 67, loss 0.1810
Training epoch 68, loss 0.0639
Validation epoch 68, loss 0.1780
Training epoch 69, loss 0.0643
Validation epoch 69, loss 0.1810
Training epoch 70, loss 0.0620
Validation epoch 70, loss 0.1706
Training epoch 71, loss 0.0611
Validation epoch 71, loss 0.1670
Training epoch 72, loss 0.0641
Validation epoch 72, loss 0.1725
Trainin

Training epoch 86, loss 0.0529
Validation epoch 86, loss 0.1558
Training epoch 87, loss 0.0546
Validation epoch 87, loss 0.1591
Training epoch 88, loss 0.0494
Validation epoch 88, loss 0.1750
Training epoch 89, loss 0.0485
Validation epoch 89, loss 0.2252
Training epoch 90, loss 0.0459
Validation epoch 90, loss 0.1541
Training epoch 91, loss 0.0474
Validation epoch 91, loss 0.1855
Training epoch 92, loss 0.0496
Validation epoch 92, loss 0.1511
Training epoch 93, loss 0.0530
Validation epoch 93, loss 0.1661
Training epoch 94, loss 0.0537
Validation epoch 94, loss 0.1806
Training epoch 95, loss 0.0546
Validation epoch 95, loss 0.1734
Training epoch 96, loss 0.0567
Validation epoch 96, loss 0.1734
Training epoch 97, loss 0.0566
Validation epoch 97, loss 0.1733
Training epoch 98, loss 0.0579
Validation epoch 98, loss 0.1789
Training epoch 99, loss 0.0521
Validation epoch 99, loss 0.1559
Training epoch 0, loss 15.1205
Validation epoch 0, loss 0.5082
Training epoch 1, loss 0.6532
Validation 

Training epoch 15, loss 0.1660
Validation epoch 15, loss 0.2176
Training epoch 16, loss 0.1651
Validation epoch 16, loss 0.2477
Training epoch 17, loss 0.1543
Validation epoch 17, loss 0.2404
Training epoch 18, loss 0.1475
Validation epoch 18, loss 0.2309
Training epoch 19, loss 0.1450
Validation epoch 19, loss 0.2239
Training epoch 20, loss 0.1413
Validation epoch 20, loss 0.2567
Training epoch 21, loss 0.1397
Validation epoch 21, loss 0.3625
Training epoch 22, loss 0.1316
Validation epoch 22, loss 0.2286
Training epoch 23, loss 0.1293
Validation epoch 23, loss 0.2064
Training epoch 24, loss 0.1270
Validation epoch 24, loss 0.2549
Training epoch 25, loss 0.1276
Validation epoch 25, loss 0.2475
Training epoch 26, loss 0.1228
Validation epoch 26, loss 0.3915
Training epoch 27, loss 0.1209
Validation epoch 27, loss 0.1948
Training epoch 28, loss 0.1140
Validation epoch 28, loss 0.2262
Training epoch 29, loss 0.1087
Validation epoch 29, loss 0.2403
Training epoch 30, loss 0.1080
Validatio

Validation epoch 43, loss 0.2161
Training epoch 44, loss 0.1151
Validation epoch 44, loss 0.2131
Training epoch 45, loss 0.1075
Validation epoch 45, loss 0.2525
Training epoch 46, loss 0.1023
Validation epoch 46, loss 0.2139
Training epoch 47, loss 0.0929
Validation epoch 47, loss 0.1809
Training epoch 48, loss 0.0924
Validation epoch 48, loss 0.1902
Training epoch 49, loss 0.0905
Validation epoch 49, loss 0.1803
Training epoch 50, loss 0.0850
Validation epoch 50, loss 0.1948
Training epoch 51, loss 0.0839
Validation epoch 51, loss 0.2053
Training epoch 52, loss 0.0808
Validation epoch 52, loss 0.2701
Training epoch 53, loss 0.0800
Validation epoch 53, loss 0.1889
Training epoch 54, loss 0.0788
Validation epoch 54, loss 0.1784
Training epoch 55, loss 0.0836
Validation epoch 55, loss 0.1770
Training epoch 56, loss 0.0825
Validation epoch 56, loss 0.1999
Training epoch 57, loss 0.0798
Validation epoch 57, loss 0.1846
Training epoch 58, loss 0.0762
Validation epoch 58, loss 0.1783
Trainin

Training epoch 72, loss 0.0555
Validation epoch 72, loss 0.1787
Training epoch 73, loss 0.0538
Validation epoch 73, loss 0.1699
Training epoch 74, loss 0.0551
Validation epoch 74, loss 0.1737
Training epoch 75, loss 0.0571
Validation epoch 75, loss 0.1779
Training epoch 76, loss 0.0637
Validation epoch 76, loss 0.2015
Training epoch 77, loss 0.0744
Validation epoch 77, loss 0.1795
Training epoch 78, loss 0.0650
Validation epoch 78, loss 0.1675
Training epoch 79, loss 0.0522
Validation epoch 79, loss 0.1740
Training epoch 80, loss 0.0488
Validation epoch 80, loss 0.1706
Training epoch 81, loss 0.0475
Validation epoch 81, loss 0.1686
Training epoch 82, loss 0.0458
Validation epoch 82, loss 0.1643
Training epoch 83, loss 0.0459
Validation epoch 83, loss 0.1638
Training epoch 84, loss 0.0475
Validation epoch 84, loss 0.1704
Training epoch 85, loss 0.0498
Validation epoch 85, loss 0.1703
Training epoch 86, loss 0.0516
Validation epoch 86, loss 0.1683
Training epoch 87, loss 0.0585
Validatio